In [1]:
# SLM Auto Importer v4 debugv1
# slimmed down from v3 - 
import openpyxl 
import string
import time
import pandas as pd
from pandas import ExcelWriter
import numpy as np
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl import cell
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Table,Paragraph
from openpyxl.utils import get_column_letter
import xlsxwriter
import matplotlib.pyplot as plot
from os import listdir, walk
from os.path import isfile, join
import tkinter as tk
from tkinter import *
import win32com
# from win32com import client
import shutil

# import win32com.client


C:\Users\jPfitsch\AppData\Local\Temp\ipykernel_44100\996283037.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
testplan_path ='//DLA-04/Shared/KAILUA PROJECTS/2024/24-004 Kaulu by Gentry ASTC - AIIC testing/Documents/TestPlan_Kaulu_ASTM_testingv1.xlsx'
test_list = pd.read_excel(testplan_path)
testnums = test_list['Test Label'] ## Determines the labels and number of excel files copied
curr_test = test_list.iloc[0]
print(curr_test)    

Area                                              NaN
Area Label                                        NaN
Test Label                                      1.1.1
Source Room                      2nd Floor Great Room
Receiving Room           1st Floor Great Room/Kitchen
Partition Type                          Floor-ceiling
Design STC                                         47
Test Date                         2024-04-04 00:00:00
Report Date                       2024-04-15 00:00:00
Source                                           A029
Recieve                                          E264
BNL                                              E263
RT                                               E140
Position1                                        E265
Position2                                        E266
Position3                                        E267
Position4                                        E268
Carpet                                           E269
SourceTap                   